In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['trainingimages']


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import random
from random import shuffle
img_w=256
img_h=256

In [3]:
 n_labels=2
def label_map(labels):
    label_map = np.zeros([img_h, img_w, n_labels])    
    for r in range(img_h):
        for c in range(img_w):
            label_map[r, c, labels[r][c]] = 1
    return label_map

In [4]:
data1=[]
data2=[]
label2=[]
label1=[]
n=0

In [5]:
TRAIN_DIR=os.listdir("../input/trainingimages/trainingimages")

def create_train_data(n,m):

    train_data=[]
    
    for patient in TRAIN_DIR[n:m]:
        pnt=os.path.join("../input/trainingimages/trainingimages",patient)
        pnt=os.listdir(pnt)
        a,b=pnt
        if a=='gt':
            gt=a
            pnt=b
        else:
            gt=b
            pnt=a
        pno=os.path.join("../input/trainingimages/trainingimages/"+patient,pnt)
        gno=os.path.join("../input/trainingimages/trainingimages/"+patient,gt)
        pnimg=os.listdir(pno)
        gtimg=os.listdir(gno)
        i=0
        for img in tqdm(pnimg):
            pnt_img_list=[cv2.resize(cv2.imread(os.path.join(pno,img),cv2.IMREAD_GRAYSCALE),(256,256))]
            
            gt_array=cv2.resize(np.clip(cv2.imread(os.path.join(gno,img)),0,1),(256,256))
                             
            data1.append(pnt_img_list)
            label1.append(label_map(gt_array))
        n=n+1
    data, label = np.array(data1).reshape(256,256,413), np.array(label1).reshape(2,256*256,413)
    
    return data,label
            
                             
            
            
          
            
train_data,train_label=create_train_data(0,2)

100%|██████████| 263/263 [00:39<00:00,  6.92it/s]


In [6]:
#train_data=train_data.reshape(4956,256,256,1)
print(train_data.shape)
train_data1=train_data.reshape(413,256,256,1)
print(train_data1.shape)

(256, 256, 413)
(413, 256, 256, 1)


In [7]:
train_label=train_label.reshape(413,65536,2)
print(train_label.shape)

(413, 65536, 2)


In [8]:
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Lambda
from keras.backend import slice
import json

img_w = 256
img_h = 256
n_labels = 2

kernel = 3



Using TensorFlow backend.


In [9]:
import keras
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D
def ESPmodule(input_img,M):
    K=4
    d=M//K
    n=3
    conv=[]
    sum1=[]
    input_img1=Conv2D(d, (1,1), padding='same', activation='relu')(input_img)
    print(input_img1.shape)
    #spatial = Lambda(lambda x: tf.split(x, num_or_size_splits=K, axis=3))(input_img1)
 
    for i in range(K):
        conv.append(Conv2D(d, (3,3), padding='same',dilation_rate=2**(i+1), activation='relu')(input_img1))
    #sum1.append(conv[0])
    """for i in range(d-1):
        sum1.append(sum1[i]+conv[i+1]) """ 
    output = keras.layers.concatenate(conv, axis = 3)
    #output1 = keras.layers.concatenate(sum1, axis = 3)
    #output=output+input_img
    print(output.shape)
    #print(output1.shape)
    return output
    
    
    
    

In [10]:
from keras.layers import Input
input_layer = Input(shape=(256, 256, 1))
x = Conv2D(16, (3,3), padding='same', activation='relu')(input_layer)
#x = Conv2D(4, (1,1), padding='same', activation='relu')(x)
y=ESPmodule(x,16)
y1=Conv2D(64, (1,1), padding='same', activation='relu')(y)
y2=ESPmodule(y1,64)
y3=ESPmodule(y2,64)
y4=ESPmodule(y3,64)
y5=Conv2D(128, (1,1), padding='same', activation='relu')(y4)
y6=ESPmodule(y5,128)
y7=ESPmodule(y6,128)
y8=ESPmodule(y7,128)
y9=ESPmodule(y8,128)
encoding_layer1=Conv2D(2, (1,1), padding='same', activation='relu')(y9)

(?, 256, 256, 4)
(?, 256, 256, 16)
(?, 256, 256, 16)
(?, 256, 256, 64)
(?, 256, 256, 16)
(?, 256, 256, 64)
(?, 256, 256, 16)
(?, 256, 256, 64)
(?, 256, 256, 32)
(?, 256, 256, 128)
(?, 256, 256, 32)
(?, 256, 256, 128)
(?, 256, 256, 32)
(?, 256, 256, 128)
(?, 256, 256, 32)
(?, 256, 256, 128)


In [11]:
print(encoding_layer1.shape)

(?, 256, 256, 2)


In [12]:
from keras.models import Model

model = Model(input_layer, encoding_layer1, name='inception_v1')
print (model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 4)  68          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 256, 256, 4)  148         conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
nb_epoch=2
batch_size=18
train_label=np.reshape(train_label,(413,256,256,2))
model.fit(train_data1, train_label, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Epoch 1/2
413/413 [==============================] - 1302s 3s/step - loss: nan - acc: 0.9563
Epoch 2/2
413/413 [==============================] - 1274s 3s/step - loss: nan - acc: 0.9906
